In [ ]:
import math
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# ─── SETTINGS ────────────────────────────────────────────────────────────────
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [ ]:
def load_model_and_tokenizer(model_name: str):
    """
    TODO:
      - Load tokenizer & model from `model_name`
      - Move model to GPU if available, choose float16 for CUDA else float32
      - Set model to eval mode
      - Return (tokenizer, model, device)
    """
    raise NotImplementedError


In [ ]:
def select_text():
    """
    TODO:
      - Pick or paste a ~100-token passage (e.g. a Wiki snippet)
      - Return (text_str)
    """
    raise NotImplementedError

In [ ]:
def tokenize_with_labels(tokenizer, text: str):
    """
    TODO:
      - Tokenize `text` with return_tensors="pt"
      - Prepare inputs and set labels = input_ids
      - Return (inputs_dict, input_len)
    """
    raise NotImplementedError

In [ ]:
def compute_peak_memory_and_loss(model, inputs, device):
    """
    TODO:
      - Reset peak memory stats if CUDA
      - Run model(**inputs) under torch.no_grad()
      - Sync CUDA if needed
      - Retrieve peak memory via torch.cuda.max_memory_allocated (in MiB)
      - Return (peak_mem_mib, loss_value)
    """
    raise NotImplementedError

In [ ]:
def compute_perplexity(loss: float):
    """
    TODO:
      - Compute and return math.exp(loss)
    """
    raise NotImplementedError

In [ ]:
def time_forward(model, inputs: Dict[str, torch.Tensor], device, num_warmup: int = 1, num_runs: int = 3):
    """
    TODO:
      1) Move all tensors in `inputs` to `device`.
      2) Warmup (no timing): under torch.no_grad(), run a small number of forward passes
         (num_warmup times) to stabilize kernels/clocks:
             _ = model(**inputs_on_device)
         If CUDA, call torch.cuda.synchronize() after warmup.
      3) Timed runs: create an empty list `latencies = []`.
         For _ in range(num_runs):
           - Record start time with time.perf_counter()
           - Under torch.no_grad(), run a forward pass:
                 _ = model(**inputs_on_device)
           - If CUDA, call torch.cuda.synchronize()
           - Record end time with time.perf_counter()
           - Append (end - start) to `latencies`.
      4) Return the average latency in seconds:
             return sum(latencies) / max(len(latencies), 1)
    """
    raise NotImplementedError

In [ ]:
import matplotlib.pyplot as plt

def sweep_context_length(
    tokenizer,
    model,
    device,
    targets=(64, 256, 1024),
    base_text=None,
    build_text_fn=None,
    warmup=1,
    runs=3,
    plot=True,
):
    """
    TODO:
      1) Determine the model's maximum context length:
         - Read `max_position_embeddings` from `model.config` (fallback to 2048).
         - Filter `targets` to those <= `max_len - 2` (leave room for special tokens).
         - If no valid targets remain, raise a ValueError.

      2) Choose a base paragraph:
         - If `base_text` is None, call `select_text()`; if unavailable, raise with a helpful message.

      3) Build texts of desired token lengths:
         - Implement a builder that repeats `base_text` until the tokenized length >= `target_tokens`.
         - Allow overriding via `build_text_fn(tokenizer, base_text, target_tokens)`.

      4) For each target length:
         - Build the text and tokenize with `tokenize_with_labels(tokenizer, text)` to get `(inputs, input_len)`.
         - Move all tensors in `inputs` to `device`.
         - Compute peak memory (MiB) and loss using `compute_peak_memory_and_loss(model, inputs, device)`.
         - Measure forward (prefill) latency using `time_forward(model, inputs, device, num_warmup=warmup, num_runs=runs)`.
         - Append `input_len`, `peak_mib`, and `latency_s` to running lists.
         - Print a concise summary line for each target.

      5) If `plot` is True, create two matplotlib plots (one per figure):
         - Peak memory (MiB) vs input tokens.
         - Forward latency (s) vs input tokens.
         - Use simple line plots with markers and grid; do not specify custom colors.

      6) Return a results dictionary with:
         - 'lengths': list of actual input token counts
         - 'peak_mib': list of peak memory values (MiB)
         - 'latency_s': list of forward latencies (s)
         - 'max_ctx': the model’s max context length
         - 'targets_used': the filtered list of targets
    """
    raise NotImplementedError

In [ ]:
from typing import Dict, Iterable, Callable, Tuple

def compare_precision_memory_perplexity(
    model_name: str,
    tokenizer,
    load_with_dtype_fn: Callable[[str], Tuple],  # expected to return (tokenizer, model, device)
    precisions: Iterable[str] = ("float16", "float32"),
    base_text: str = None,
) -> Dict[str, Dict[str, float]]:
    """
    TODO:
      1) Prepare a short evaluation paragraph:
         - If `base_text` is None, call `select_text()` to get a default paragraph.

      2) Tokenize once with the provided `tokenizer`:
         - Use `tokenize_with_labels(tokenizer, text)` to get `(inputs_s, Ls)`.

      3) For each precision in `precisions`:
         - Reload the model with that dtype via: tok_p, mod_p, dev_p = load_with_dtype_fn(precision)
           (Assume this function *honors* dtype; on CPU it may force float32.)
         - Move pre-tokenized inputs to the appropriate device:
               inp = {k: v.to(dev_p) for k, v in inputs_s.items()}
         - Measure peak memory (MiB) and loss:
               peak_mib, loss_val = compute_peak_memory_and_loss(mod_p, inp, dev_p)
         - Convert loss to perplexity:
               ppl = compute_perplexity(loss_val)
         - Save into a results dict keyed by precision:
               results[precision] = {"peak_mib": peak_mib, "ppl": ppl}

      4) Return the results dict.

    Returns:
      Dict[str, Dict[str, float]] like:
        {"float16": {"peak_mib": 123.4, "ppl": 15.2}, "float32": {...}}
    """
    raise NotImplementedError

In [ ]:
def sweep_batch_memory(
    tokenizer,
    model,
    device,
    batch_sizes=(1, 2, 4),
    base_text=None,
    target_len=128,
    plot=True,
):
    """
    TODO:
      1) Choose a base paragraph:
         - If `base_text` is None, call `select_text()` to get a default paragraph.

      2) Implement a helper to build batched inputs:
         - Given (tokenizer, text, bs, max_len), tokenize a list of the same `text`
           repeated `bs` times with:
               return_tensors='pt', padding='longest', truncation=True, max_length=target_len
         - Set enc['labels'] = enc['input_ids'].clone()

      3) For each batch size in `batch_sizes`:
         - Build the batch with the helper.
         - Move all tensors to `device`.
         - Measure peak memory (MiB) using `compute_peak_memory_and_loss(model, enc, device)`.
         - Record (bs, peak_mib) and print a concise summary line.

      4) If `plot` is True, produce a single matplotlib figure of:
         - x = batch size
         - y = peak MiB
         - Use a simple line plot with markers and grid; do not set custom colors.

      5) Return a results dictionary:
         {
           "batch_sizes": [...],
           "peak_mib": [...],
           "target_len": target_len,
         }
    """
    raise NotImplementedError

In [ ]:
def start():
    if "load_with_dtype" not in globals():
        def load_with_dtype(dtype_str: str):
            tok_p, mod_p, dev_p = load_model_and_tokenizer(MODEL_NAME)
            return tok_p, mod_p, dev_p


    # --- 1) Load model/tokenizer ---
    tokenizer, model, device = load_model_and_tokenizer(MODEL_NAME)
    print(f"[load] device={device}")

    # --- 2) Select & tokenize a baseline paragraph ---
    text = select_text()
    inputs, input_len = tokenize_with_labels(tokenizer, text)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    print(f"[tokenize] tokens={input_len}")

    # --- 3) Baseline: peak memory & loss ---
    peak_mib, loss = compute_peak_memory_and_loss(model, inputs, device)
    ppl = compute_perplexity(loss)
    fwd_lat = time_forward(model, inputs, device)
    print(f"[baseline] peak={peak_mib:.1f} MiB | loss={loss:.3f} | ppl={ppl:.3f} | fwd={fwd_lat:.3f}s")

    # --- A) Context-length sweep ---
    ctx_results = sweep_context_length(
        tokenizer=tokenizer,
        model=model,
        device=device,
    )

    # --- B) Precision comparison ---
    prec_results = compare_precision_memory_perplexity(
        model_name=MODEL_NAME,
        tokenizer=tokenizer,
        load_with_dtype_fn=load_with_dtype,
        precisions=("float16", "float32"),
        base_text=text,
    )

    # --- C) Batch-size sweep ---
    batch_results = sweep_batch_memory(
        tokenizer=tokenizer,
        model=model,
        device=device,
    )

    return {
        "baseline": {
            "tokens": int(input_len),
            "peak_mib": float(peak_mib),
            "loss": float(loss),
            "ppl": float(ppl),
            "forward_latency_s": float(fwd_lat),
        },
        "context_sweep": ctx_results,
        "precision_compare": prec_results,
        "batch_sweep": batch_results,
    }


In [ ]:
start()